# Post process tool used to generate path overlays on top of floor map
- used to visually compare predicted and ground truth path x,y, and floor values


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Choose site location to simplify and speed up the output
- Floor mapping is used based on the input file
- options have been set based on the input and output of the LSTM implementation and the LGBM implementation

In [ ]:
from pathlib import Path

floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2,
             "F4":3, "F5":4, "F6":5, "F7":6,"F8":7,"F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5,
             "7F":6, "8F": 7, "9F":8}

counter_map = {-2:"B2", -1:"B1", 0:"F1", 1:"F2", 2:"F3",
             3:"F4", 4:"F5", 5:"F6"}

def split_col(df):
    df = pd.concat([
        df['site_path_timestamp'].str.split('_', expand=True) \
        .rename(columns={0:'site',
                         1:'path',
                         2:'timestamp'}),
        df
    ], axis=1).copy()
    return df

sub_df = pd.read_csv('../input/light-gbm-indoorloc/submission.csv')
sub_df = split_col(sub_df[['site_path_timestamp','floor','x','y']]).copy()

#predTrue = pd.read_csv('../input/predictedvtruth/sample_output.csv')
predTrue = pd.read_csv('../input/light-gbm-indoorloc/sample_output.csv')

# lstm site = 5a0546857ecc773753327266
# lgbm site = 5d27096c03f801723c31e5e0

#def generate_target_sites(sub_df):
#    return sorted(sub_df['site'].unique())

def generate_site_floors_dict(sub_df):
    #sites = generate_target_sites(sub_df)
    #site = '5a0546857ecc773753327266'
    site='5d27096c03f801723c31e5e0'
    #site = '5a0546857ecc773753327266'
    site_floors_dict = {}
    
    #for site in sites:
    site_path = Path('/kaggle/input/indoor-location-navigation/train') / site
    site_floors_dict[site] = [path.name for path in site_path.glob('*')]
    return site_floors_dict

#all_sites = generate_target_sites(sub_df)
site_floors_dict = generate_site_floors_dict(sub_df)
site_floors_dict



In [ ]:
predTrue

In [ ]:
def plot_sites(sites, sub1_df):#, sub2_df, sub3_df):
    num_floors = 0
    for site in sites:
        num_floors += len(site_floors_dict[site])

    fig, ax = plt.subplots(num_floors, 2, figsize=(8, 50))

    idx = 0
    for site in sites:
        floors = site_floors_dict[site]

        for floor in floors:
            print(site, " : ", floor, " : ", )
            #plot_preds(ax[idx], "raw submission", site, floor, sub1_df, show_preds=True)#, train_waypoints, show_preds=True)
            plot_preds_truth(ax[idx][0], "predictions", site, floor, sub1_df, show_preds=True, show_train=False)#, train_waypoints, show_preds=True)
            plot_preds_truth(ax[idx][1], "ground truth", site, floor, sub1_df, show_preds=False, show_train=True)
            ##plot_preds(ax[idx][2], "snap_to_grid", site, floor, sub3_df, train_waypoints, show_preds=True)
            idx += 1
    fig.savefig('compare.png')
    plt.show()    

In [ ]:
import json

def plot_preds(
    ax,
    context_text,
    site,
    floorNo,
    sub=None,
    show_preds=True,
    fix_labels=True,
    true_locs=None,
    base="../input/indoor-location-navigation",
    show_train=False,        
    map_floor=None
):
    """
    Plots predictions on floorplan map.
    
    map_floor : use a different floor's map
    """
    #map_floor = {-2:"B2", -1:"B1", 0:"F1", 1:"F2", 2:"F3",
    #         3:"F4", 4:"F5", 5:"F6"}
    
    floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2,
             "F4":3, "F5":4, "F6":5, "F7":6,"F8":7,"F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5,
             "7F":6, "8F": 7, "9F":8}
    
    if map_floor is None:
        map_floor = floorNo
    # Prepare width_meter & height_meter (taken from the .json file)
    floor_plan_filename = f"{base}/metadata/{site}/{map_floor}/floor_image.png"
    json_plan_filename = f"{base}/metadata/{site}/{map_floor}/floor_info.json"
    
    with open(json_plan_filename) as json_file:
        json_data = json.load(json_file)

    width_meter = json_data["map_info"]["width"]
    height_meter = json_data["map_info"]["height"]

    floor_img = plt.imread(f"{base}/metadata/{site}/{map_floor}/floor_image.png")


    ax.imshow(floor_img)

    if show_train:
        true_locs = true_locs.query('site == @site and floorNo == @map_floor').copy()
        true_locs["x_"] = true_locs["x"] * floor_img.shape[0] / height_meter
        true_locs["y_"] = (
            true_locs["y"] * -1 * floor_img.shape[1] / width_meter
        ) + floor_img.shape[0]
        true_locs.query("site == @site and floorNo == @map_floor").groupby("path").plot(
            x="x_",
            y="y_",
            style="+",
            ax=ax,
            label="train waypoint location",
            color="grey",
            alpha=0.5,
        )

    if show_preds:
        sub = sub[(sub['site']==site) & (sub['floor']==floor_map[floorNo])].copy()
        #sub = sub[sub['site']==site]
        #sub = sub[sub['floor']==floorNo]
        
        sub["x_"] = sub["x"] * floor_img.shape[0] / height_meter
        sub["y_"] = (sub["y"] * -1 * floor_img.shape[1] / width_meter) + floor_img.shape[0]
        
        for path, path_data in sub.groupby("path"):            
            path_data.plot(
                x="x_",
                y="y_",
                style=".-",
                ax=ax,
                title="context_text",#f"{context_text} - {site} - floor - {floorNo}",
                alpha=1,
                label=path,
            )
    if fix_labels:
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        ax.legend(
            by_label.values(), by_label.keys(), loc="center left", bbox_to_anchor=(1, 0.5)
        )
    return

In [ ]:
import json

def plot_preds_truth(
    ax,
    context_text,
    site,
    floorNo,
    sub=None,
    show_preds=True,
    fix_labels=True,
    true_locs=None,
    base="../input/indoor-location-navigation",
    show_train=False,        
    map_floor=None
):
    """
    Plots predictions on floorplan map.
    
    map_floor : use a different floor's map
    """
    #map_floor = {-2:"B2", -1:"B1", 0:"F1", 1:"F2", 2:"F3",
    #         3:"F4", 4:"F5", 5:"F6"}
    
    floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2,
             "F4":3, "F5":4, "F6":5, "F7":6,"F8":7,"F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5,
             "7F":6, "8F": 7, "9F":8}
    
    if map_floor is None:
        map_floor = floorNo
    # Prepare width_meter & height_meter (taken from the .json file)
    floor_plan_filename = f"{base}/metadata/{site}/{map_floor}/floor_image.png"
    json_plan_filename = f"{base}/metadata/{site}/{map_floor}/floor_info.json"
    
    with open(json_plan_filename) as json_file:
        json_data = json.load(json_file)

    width_meter = json_data["map_info"]["width"]
    height_meter = json_data["map_info"]["height"]

    floor_img = plt.imread(f"{base}/metadata/{site}/{map_floor}/floor_image.png")


    ax.imshow(floor_img)

    if show_train:
        #sub = sub[(sub['site']==site) & (sub['floor']==floor_map[floorNo])].copy()
        sub = sub[(sub['fTrue']==floor_map[floorNo])].copy()
        #sub = sub[sub['site']==site]
        #sub = sub[sub['floor']==floorNo]
        
        sub["x_"] = sub["xPred"] * floor_img.shape[0] / height_meter
        sub["y_"] = (sub["yPred"] * -1 * floor_img.shape[1] / width_meter) + floor_img.shape[0]
        sub["x_t"] = sub["xTrue"] * floor_img.shape[0] / height_meter
        sub["y_t"] = (sub["yTrue"] * -1 * floor_img.shape[1] / width_meter) + floor_img.shape[0]
        
        count = 0
        for path, path_data in sub.groupby("path"):            
            if count < 10:
                path_data.plot(
                    x="x_t",
                    y="y_",
                    style=".-",
                    ax=ax,
                    title=f"{context_text}",#} - {site} - floor - {floorNo}",
                    alpha=1,
                    #label=path,
                )       
            count+=1

    if show_preds:
        #sub = sub[(sub['site']==site) & (sub['floor']==floor_map[floorNo])].copy()
        sub = sub[(sub['fTrue']==floor_map[floorNo])].copy()
        #sub = sub[sub['site']==site]
        #sub = sub[sub['floor']==floorNo]
        
        sub["x_"] = sub["xPred"] * floor_img.shape[0] / height_meter
        sub["y_"] = (sub["yPred"] * -1 * floor_img.shape[1] / width_meter) + floor_img.shape[0]
        sub["x_t"] = sub["xTrue"] * floor_img.shape[0] / height_meter
        sub["y_t"] = (sub["yTrue"] * -1 * floor_img.shape[1] / width_meter) + floor_img.shape[0]
        
        count = 0
        for path, path_data in sub.groupby("path"):            
            if count < 10:
                path_data.plot(
                    x="x_",
                    y="y_",
                    style=".-",
                    ax=ax,
                    title=f"{context_text}",#f"{context_text} - {site} - floor - {floorNo}",
                    alpha=1,
                    #label=path,
                )       
            count+=1
        
    if fix_labels:
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        ax.legend(
            by_label.values(), by_label.keys(), loc="center left", bbox_to_anchor=(1, 0.5)
        )
    return

In [ ]:
#sub_df[sub_df['site']=='5a0546857ecc773753327266'].head()
floorNo = 0
#site = '5a0546857ecc773753327266'
site = '5d27096c03f801723c31e5e0'
#sub = sub_df[(sub_df['site']==site) & (sub_df['floor']==floorNo)].copy()#'site == @site and floor == @floor').copy()
#sub
#sub = sub_df[sub_df['site']==site]
#sub = sub_df[sub_df['floor']==floorNo]

#sub["x_"] = sub["x"] * floor_img.shape[0] / height_meter
#sub["y_"] = (sub["y"] * -1 * floor_img.shape[1] / width_meter) + floor_img.shape[0]

#for path, path_data in sub.groupby("path"):
#    print(path_data)
##sub["x_"] = sub["x"] * floor_img.shape[0] / height_meter
##sub["y_"] = (sub["y"] * -1 * floor_img.shape[1] / width_meter) + floor_img.shape[0] 
##for path, path_data in sub.query("site == @site and floor == @floorNo").groupby("path"):
##    path_data.plot(x="x_",y="y_",style=".-",ax=ax,title=f"{context_text} - {site} - floor - {floorNo}",alpha=1,label=path,)
#sub_df.head()

# Data is formated and plotted

In [ ]:
#sites=['5a0546857ecc773753327266']
sites = ['5d27096c03f801723c31e5e0']
#plot_sites(sites, sub_df)#, processed_sub_df1, processed_sub_df2)
plot_sites(sites, predTrue)
